In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import time

# ============================================================
# PATHS
# ============================================================
BASE ="/Volumes/workspace/default/pysparkassesment"
input_dir  = f"{BASE}/input"
output_dir = f"{BASE}/output"
error_dir  = f"{BASE}/error"

print("Loaded Paths:")
print("INPUT :", input_dir)
print("OUTPUT:", output_dir)
print("ERROR :", error_dir)


try:
    # ============================================================
    # 1️⃣ READ ONLY LAST 24 HOURS FILES
    # ============================================================
    files = dbutils.fs.ls(input_dir)
    cutoff = datetime.now() - timedelta(hours=24)

    recent_files = [
        f.path for f in files
        if datetime.fromtimestamp(f.modificationTime/1000) >= cutoff
    ]

    print("\n📥 Reading Recent Files:")
    for f in recent_files:
        print(f)

    if not recent_files:
        raise Exception("❌ No recent files found in last 24 hours")

    df = spark.read.option("header", True).csv(recent_files)

    # ============================================================
    # 2️⃣ SAFE CAST NUMBERS
    # ============================================================
    df = (
        df.withColumn("PriceStartD", F.expr("try_cast(PriceStart AS double)"))
          .withColumn("PriceEndD",   F.expr("try_cast(PriceEnd AS double)"))
    )

    # ============================================================
    # 3️⃣ ERROR REASON LOGIC
    # ============================================================
    df = df.withColumn(
        "ErrorReason",
        F.concat_ws("; ",
            F.when(F.col("PriceStart").isNull(), "Missing PriceStart"),
            F.when(F.col("PriceEnd").isNull(), "Missing PriceEnd"),
            F.when(F.col("PriceStart").isNotNull() & F.col("PriceStartD").isNull(), "Invalid PriceStart Format"),
            F.when(F.col("PriceEnd").isNotNull()   & F.col("PriceEndD").isNull(),   "Invalid PriceEnd Format"),
            F.when(F.col("PriceStartD") <= 0,"PriceStart <= 0"),
            F.when(F.col("PriceEndD") <= 0, "PriceEnd <= 0"),
            F.when(~F.col("Month").rlike("^[0-9]{4}-[0-9]{2}$"), "Invalid Month Format")
        )
    )

    # ============================================================
    # 4️⃣ CALCULATE RETURN PERCENTAGE
    # ============================================================
    df = df.withColumn(
        "ReturnPct",
        F.when(
            (F.col("PriceStartD") > 0) & (F.col("PriceEndD").isNotNull()),
            F.round(((F.col("PriceEndD") - F.col("PriceStartD")) / F.col("PriceStartD")) * 100, 2)
        )
    )

    # UNREALISTIC RETURN ERROR
    df = df.withColumn(
        "ErrorReason",
        F.when(F.col("ReturnPct") > 200,
               F.concat_ws("; ", F.col("ErrorReason"), F.lit("Unrealistic Return > 200%")))
        .otherwise(F.col("ErrorReason"))
    )

    # ============================================================
    # 5️⃣ SPLIT GOOD & BAD
    # ============================================================
    df_bad = df.filter(F.length("ErrorReason") > 0)
    df_good = df.filter(F.length("ErrorReason") == 0)
    df_good = df_good.withColumn("ErrorReason", F.lit("No Error Found"))

    # SAVE BAD DATA
    ts = str(int(time.time() * 1000))
    bad_path = f"{error_dir}/Bad_Data_{ts}.csv"
    df_bad.write.mode("overwrite").csv(bad_path, header=True)

    # DISPLAY BAD DATA
    print("📌 BAD DATA:")
    display(df_bad)

    # ============================================================
    # 6️⃣ GOOD DATA PROCESSING (PER MONTH)
    # ============================================================
    df_good = df_good.withColumn("PriceChange", F.col("PriceEndD") - F.col("PriceStartD"))
    df_good = df_good.withColumn("Trend", F.when(F.col("PriceEndD") > F.col("PriceStartD"), "Gain").otherwise("Loss"))

    # Calculate mean & std per month
    month_stats = df_good.groupBy("Month").agg(
        F.mean("ReturnPct").alias("mean"),
        F.stddev("ReturnPct").alias("std")
    )

    # Join back stats to df_good
    df_good = df_good.join(month_stats, on="Month", how="left")
    df_good = df_good.withColumn("Threshold", F.col("mean") + 2 * F.col("std"))
    df_good = df_good.withColumn("IsOutlier", F.col("ReturnPct") > F.col("Threshold"))

    # Ranking per month
    w = Window.partitionBy("Month").orderBy(F.col("ReturnPct").desc())
    df_good = df_good.withColumn("Rank", F.rank().over(w))

    # DISPLAY GOOD DATA
    print("📌 GOOD DATA:")
    display(df_good)

    # ============================================================
    # BEST & WORST PER MONTH
    # ============================================================
    best_df_list = []
    worst_df_list = []

    months = [row["Month"] for row in df_good.select("Month").distinct().collect()]

    for m in months:
        month_df = df_good.filter(F.col("Month") == m)
        best_row  = month_df.orderBy(F.col("ReturnPct").desc()).first()
        worst_row = month_df.orderBy(F.col("ReturnPct").asc()).first()
        common_cols = df_good.columns

        best_df_list.append(spark.createDataFrame([best_row.asDict()])[common_cols])
        worst_df_list.append(spark.createDataFrame([worst_row.asDict()])[common_cols])

    # Safe union
    best_df = best_df_list[0]
    for df in best_df_list[1:]:
        best_df = best_df.union(df)

    worst_df = worst_df_list[0]
    for df in worst_df_list[1:]:
        worst_df = worst_df.union(df)

    # DISPLAY BEST & WORST
    print("🌟 BEST PER MONTH:")
    display(best_df)

    print("❌ WORST PER MONTH:")
    display(worst_df)

    # ============================================================
    # FINAL UNION (BEST + WORST + ALL GOOD)
    # ============================================================
    final_df = best_df.union(worst_df).union(df_good)

    # DISPLAY FINAL DATA
    print("📊 FINAL DATAFRAME:")
    display(final_df)

    # ============================================================
    # 7️⃣ SAVE FINAL OUTPUT (CSV)
    # ============================================================
    out_csv = f"{output_dir}/Transformed_{ts}.csv"
    final_df.write.mode("overwrite").csv(out_csv, header=True)

    print("\n🎉 SUCCESS")
    print("CSV :", out_csv)
    print("ERROR :", bad_path)

except Exception as e:
    print("🔥 PIPELINE FAILED:", e)
    print("Please check the error logs for details.")


Loaded Paths:
INPUT : /Volumes/workspace/default/pysparkassesment/input
OUTPUT: /Volumes/workspace/default/pysparkassesment/output
ERROR : /Volumes/workspace/default/pysparkassesment/error

📥 Reading Recent Files:
dbfs:/Volumes/workspace/default/pysparkassesment/input/pyspark_baddata_alltypes.csv
dbfs:/Volumes/workspace/default/pysparkassesment/input/pyspark_stock1.csv
dbfs:/Volumes/workspace/default/pysparkassesment/input/pyspark_stock_1000rows.csv
dbfs:/Volumes/workspace/default/pysparkassesment/input/pyspark_stock_100rows.csv
dbfs:/Volumes/workspace/default/pysparkassesment/input/pyspark_stock_multimonth.csv
📌 BAD DATA:


Stock,Sector,Month,PriceStart,PriceEnd,PriceStartD,PriceEndD,ErrorReason,ReturnPct
BABA,Retail,2025-02,94,439,94.0,439.0,; Unrealistic Return > 200%,367.02
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
BABA,Retail,2025-02,94,439,94.0,439.0,; Unrealistic Return > 200%,367.02
BABA,Retail,2025-02,94,439,94.0,439.0,; Unrealistic Return > 200%,367.02
BABA,Retail,2025-02,94,439,94.0,439.0,; Unrealistic Return > 200%,367.02


📌 GOOD DATA:


Month,Stock,Sector,PriceStart,PriceEnd,PriceStartD,PriceEndD,ErrorReason,ReturnPct,PriceChange,Trend,mean,std,Threshold,IsOutlier,Rank
2025-01,TSLA,Automotive,600.0,880.0,600.0,880.0,No Error Found,46.67,280.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,true,1
2025-01,AAPL,Technology,120.5,175.2,120.5,175.2,No Error Found,45.39,54.69999999999999,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,2
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3


🌟 BEST PER MONTH:


...

❌ WORST PER MONTH:


Month,Stock,Sector,PriceStart,PriceEnd,PriceStartD,PriceEndD,ErrorReason,ReturnPct,PriceChange,Trend,mean,std,Threshold,IsOutlier,Rank
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182
2025-03,BABA,Retail,2102,1873,2102.0,1873.0,No Error Found,-10.89,-229.0,Loss,2.505625000000001,6.989190728239879,16.484006456479758,false,162
2025-06,AAPL,Tech,2092,1800,2092.0,1800.0,No Error Found,-13.96,-292.0,Loss,19.306648936170216,22.339435717483113,63.985520371136445,false,177
2025-02,BAJAJ-AUTO,Auto,451,295,451.0,295.0,No Error Found,-34.59,-156.0,Loss,-5.04916666666667,15.616598563654119,26.184030460641566,false,166
2025-04,MSFT,Tech,2493,2247,2493.0,2247.0,No Error Found,-9.87,-246.0,Loss,7.407839195979903,12.258395591555281,31.924630379090466,false,179
2025-05,RELIANCE,Energy,1576,1461,1576.0,1461.0,No Error Found,-7.3,-115.0,Loss,20.529298245614058,25.683453097939456,71.89620444149297,false,154


📊 FINAL DATAFRAME:


Month,Stock,Sector,PriceStart,PriceEnd,PriceStartD,PriceEndD,ErrorReason,ReturnPct,PriceChange,Trend,mean,std,Threshold,IsOutlier,Rank
2025-01,TSLA,Automotive,600.0,880.0,600.0,880.0,No Error Found,46.67,280.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,true,1
2025-01,AAPL,Technology,120.5,175.2,120.5,175.2,No Error Found,45.39,54.69999999999999,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,2
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3



🎉 SUCCESS
CSV : /Volumes/workspace/default/pysparkassesment/output/Transformed_1763536744077.csv
ERROR : /Volumes/workspace/default/pysparkassesment/error/Bad_Data_1763536744077.csv


In [0]:
from pyspark.sql import functions as F

df_good = df_good.withColumn(
    "PerformanceLevel",
    F.when(F.col("ReturnPct") > 20, "Excellent")
     .when((F.col("ReturnPct") >= 5) & (F.col("ReturnPct") <= 20), "Moderate")
     .otherwise("Poor")
)
display(df_good)

Month,Stock,Sector,PriceStart,PriceEnd,PriceStartD,PriceEndD,ErrorReason,ReturnPct,PriceChange,Trend,mean,std,Threshold,IsOutlier,Rank,PerformanceLevel
2025-01,TSLA,Automotive,600.0,880.0,600.0,880.0,No Error Found,46.67,280.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,true,1,Excellent
2025-01,AAPL,Technology,120.5,175.2,120.5,175.2,No Error Found,45.39,54.69999999999999,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,2,Excellent
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3,Excellent
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3,Excellent
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3,Excellent
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3,Excellent
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3,Excellent
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3,Excellent
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3,Excellent
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3,Excellent


In [0]:
# ============================================================
# 7️⃣ SAVE FINAL OUTPUT (CSV)
# ============================================================
'''
out_csv = f"{output_dir}/Transformed_{ts}.csv"
final_df.write.mode("overwrite").csv(out_csv, header=True)
'''

# Display the final CSV in notebook
print("\n📄 DISPLAYING FINAL CSV OUTPUT:")
final_output_df = spark.read.option("header", True).csv(out_csv)
display(final_output_df)

# Display the bad/error CSV in notebook
print("\n📄 DISPLAYING ERROR CSV:")
error_output_df = spark.read.option("header", True).csv(bad_path)
display(error_output_df)

print("\n🎉 SUCCESS")
print("CSV saved to:", out_csv)
print("Error CSV saved to:", bad_path)



📄 DISPLAYING FINAL CSV OUTPUT:


Month,Stock,Sector,PriceStart,PriceEnd,PriceStartD,PriceEndD,ErrorReason,ReturnPct,PriceChange,Trend,mean,std,Threshold,IsOutlier,Rank
2025-01,TSLA,Automotive,600.0,880.0,600.0,880.0,No Error Found,46.67,280.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,true,1
2025-01,AAPL,Technology,120.5,175.2,120.5,175.2,No Error Found,45.39,54.69999999999999,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,2
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3
2025-01,WMT,Staples,1024,1413,1024.0,1413.0,No Error Found,37.99,389.0,Gain,6.622095238095241,19.71875243271866,46.05960010353256,false,3



📄 DISPLAYING ERROR CSV:


Stock,Sector,Month,PriceStart,PriceEnd,PriceStartD,PriceEndD,ErrorReason,ReturnPct
BABA,Retail,2025-02,94,439,94.0,439.0,; Unrealistic Return > 200%,367.02
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
TSLA,Auto,2025-01,139,491,139.0,491.0,; Unrealistic Return > 200%,253.24
BABA,Retail,2025-02,94,439,94.0,439.0,; Unrealistic Return > 200%,367.02
BABA,Retail,2025-02,94,439,94.0,439.0,; Unrealistic Return > 200%,367.02
BABA,Retail,2025-02,94,439,94.0,439.0,; Unrealistic Return > 200%,367.02



🎉 SUCCESS
CSV saved to: /Volumes/workspace/default/pysparkassesment/output/Transformed_1763536744077.csv
Error CSV saved to: /Volumes/workspace/default/pysparkassesment/error/Bad_Data_1763536744077.csv


In [0]:
total_investment = df_good.agg(F.sum("PriceEndD")).collect()[0][0]
sector_window = Window.partitionBy("Sector")
df_good = df_good.withColumn("Sectorexposure",F.round((F.sum("PriceEndD").over(sector_window)/total_investment)*100,2))
print("sector exposure added:")
display(df_good)


sector exposure added:


Month,Stock,Sector,PriceStart,PriceEnd,PriceStartD,PriceEndD,ErrorReason,ReturnPct,PriceChange,Trend,mean,std,Threshold,IsOutlier,Rank,PerformanceLevel,Sectorexposure
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
2025-01,BAJAJ-AUTO,Auto,336,245,336.0,245.0,No Error Found,-27.08,-91.0,Loss,6.622095238095241,19.71875243271866,46.05960010353256,false,182,Poor,19.44
